In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install --quiet git+https://github.com/huggingface/transformers.git@5c00918681d6b4027701eb46cea8f795da0d4064
!pip install --quiet sentencepiece==0.1.95
!pip install --quiet datasets==1.0.2
!pip install --quiet torch==1.7.1
!pip install --quiet fastt5==0.0.4
#!pip install --quiet transformers==4.2.2
!pip install --quiet pytorch-lightning==1.1.3
#!pip install --quiet tokenizers==0.9.4
!pip install --quiet torchtext==0.8.1
!pip install --quiet tqdm==4.64.0
!pip install transformers -U --quiet
!pip install tokenizers --quiet
!pip install --quiet PyPDF2
!pip install --quiet PyPaperBot
!pip install --quiet ipython-autotime

In [ ]:
#load base dataset
from datasets import load_dataset
from pprint import pprint
import os
train_data = load_dataset('squad', split='train')
validate_data = load_dataset('squad', split='validation')

sample_validation_dataset = next(iter(validate_data))
# pprint (sample_validation_dataset)

In [ ]:
#creat train set and validation set
import pandas as pd
pd.set_option("display.max_colwidth", -1)
from tqdm.notebook import tqdm
df_train = pd.DataFrame(columns = ['context','answers', 'question'])
df_validation = pd.DataFrame(columns = ['context', 'answers', 'question'])
# print (df_validation)
# print (df_train)

In [ ]:
counter_train= 0

for index,val in enumerate(tqdm(train_data)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words > 5:
        df_train.loc[counter_train]= [passage] + [answer] + [question]
        counter_train = counter_train + 1

print ("counter train dataset: ",counter_train)


100%|██████████| 87599/87599 [10:27<00:00, 139.59it/s]

count_long train dataset:  8935
count_short train dataset:  78664
time: 10min 27s (started: 2022-10-03 04:32:57 +00:00)


In [ ]:
counter_val = 0
df_train = pd.DataFrame(columns=['passage','answer','question'])
for index,val in enumerate(tqdm(validate_data)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words > 5:
        df_validation.loc[counter_val]= [passage] + [answer] + [question]
        counter_val = counter_val + 1
print ("count of validate dataset: ",counter_val)


100%|██████████| 10570/10570 [00:25<00:00, 408.71it/s]

count_long validate dataset:  918
count_short validate dataset:  9652
time: 25.9 s (started: 2022-10-03 04:43:25 +00:00)


In [ ]:
#shuffling the data
from sklearn.utils import shuffle
df_train = shuffle(df_train)
df_validation = shuffle(df_validation)
print (df_train.shape)
print (df_validation.shape)

(78664, 3)
(9652, 3)
time: 155 ms (started: 2022-10-03 04:43:50 +00:00)


In [ ]:
train_save_path = '/content/gdrive/My Drive/Colab_Notebooks/t5/dataset/squad_t5_train.csv'#\AgOCQs\model\t5\dataset\squad_t5_train.csv'
validation_save_path = '/content/gdrive/My Drive/Colab_Notebooks/t5/dataset/squad_t5_train.csv'#\AgOCQs\model\t5\dataset\squad_t5_val.csv'
df_train.to_csv(train_save_path, index = False)
df_validation.to_csv(validation_save_path, index = False)

time: 3.73 s (started: 2022-10-03 04:43:51 +00:00)


In [ ]:
train_file_path = '/content/gdrive/My Drive/Colab_Notebooks/t5/dataset/squad_t5_train.csv'
validation_file_path = '/content/gdrive/My Drive/Colab_Notebooks/t5/dataset/squad_t5_val.csv'

time: 819 µs (started: 2022-10-03 04:43:54 +00:00)


In [ ]:
#extract pdf from google scholar
#beware that the pdfs are not always the full version and may have missing pages and will differ from mine due to 
#different versions and times of download
 query = "Covid-19"  # Enter your search query here
!python -m PyPaperBot --query="Covid-19" --scholar-pages=50 --min-year=2020  --dwn-dir="/content/gdrive/MyDrive/Colab_Notebooks/newPDFs" --scihub-mirror="https://sci-hub.ee"


In [ ]:
import PyPDF2
path="/content/gdrive/My Drive/Colab_Notebooks/papers/"
for file in os.listdir(path):
    if file.endswith(".pdf"):
        fpath=os.path.join(path, file)
        pdffileobj=open(fpath,'rb')
        pdfreader=PyPDF2.PdfReader(pdffileobj)
        newfpath=re.sub(".pdf",".txt",fpath)
        with open(newfpath,"w") as file:
            for page in pdfreader.pages:
                input_text = ""
                text=page.extract_text()
                intro = text.lower().find('introduction')
                ref_text = text[intro+len('introduction'):]
                # introTxt= text.replace(ref_text, '')
                input_text+=ref_text
                relwks = text.lower().find('related works')
                rel_work = text[relwks+len('related works'):]
                # rel_work= text.replace(rel_work, '')
                input_text+=rel_work
                mthd = text.lower().find('method')
                mthd_text = text[mthd+len('method'):]
                # MethdTxt= text.replace(mthd_text, '')
                input_text+=mthd_text
                file.write(input_text)

In [ ]:

import pandas as pd
import numpy as np
import os
import sys
import re
import spacy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import tokenize
from operator import itemgetter
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
from spacy.lang.tokenizer_exceptions import URL_PATTERN
nlp = spacy.load("en_core_web_sm")
import PyPDF2
from transformers import T5ForConditionalGeneration,T5Tokenizer
pd.set_option("display.max_colwidth", -1)
#extract sentences
def extract_sentences(file):
    content = nlp(str(file))
    return [x.text for x in content.sents]
newtext =[]
outText = []
text = {}
dict_of_articles={}
text_path = "gdrive/My Drive/Colab_Notebooks/papers/inputText"
files = (file for file in os.listdir(text_path)
         if os.path.isfile(os.path.join(text_path, file)))
for file in files:
  key_name = os.path.splitext(file)[0]
  mypdf = open(os.path.join(text_path, file), "r")
  T = mypdf.readlines()
  final = [i.replace('\r\n',' ').replace('b\r\n', '').replace('\n', '')  for i in T]
  outText.append(T)
  newtext.append(final)

# df = pd.DataFrame(newtext).T


In [ ]:
sentText= extract_sentences(outText)
final = [i.replace("\n'"," ") for i in sentText]
#result = re.sub(r'http\S+', '', str(final))
newFinal =[re.sub("[\(%\[].*?[\)\n]]", " ", i) for i in final]
newFinal =[re.sub(r'http\S+', '', i) for i in newFinal]
df2 =pd.DataFrame(final, columns= ['sentences'])
df3 =pd.DataFrame(newFinal, columns= ['sentences'])

for i in range(0, len(df2)):
    df2.at[i, 'sentences_new'] = re.sub(r'\b(\d+\w{1,2})\b'," ",str(df2.loc[i,'sentences']))
    df2.at[i, 'sentences_new'] = re.sub(r'[\d+]'," ",str(df2.loc[i,'sentences_new']))
    df2.at[i, 'sentences_new'] = re.sub(r'[^\w\s]'," ",str(df2.loc[i,'sentences_new']))
    #df2.at[i, 'sentences_new'] = re.sub(r'\b\w{1}\b'," ",str(df2.loc[i,'sentences_new']))
df2['text'] = df2['sentences_new'].str.replace(r"\s*https?:'//\S+(\s+|$)", " ").str.strip()
df2['text2'] = df2['text'].str.replace(r"ufeff", " ").str.strip()
df2.head()

In [ ]:
df4= df2.dropna()
del df2['sentences_new']
del df2['text']

In [ ]:
import argparse
from itertools import chain
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from termcolor import colored
import textwrap
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
pl.seed_everything(42)

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
import copy
class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_inp=512,max_len_out=96):
        self.path = filepath

        self.passage_column = "context"
        #self.answer = "answer"
        self.question = "question"

        # self.data = pd.read_csv(self.path)
        self.data = pd.read_csv(self.path,nrows=1000)

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.skippedcount =0
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for idx in tqdm(range(len(self.data))):
            passage,target = self.data.loc[idx, self.passage_column], self.data.loc[idx, self.question]

            input_ = "context: %s </s>" % (passage)
            target = "question: %s </s>" % (str(target))

            # get encoding length of input. If it is greater than self.max_len skip it
            test_input_encoding = self.tokenizer.encode_plus(input_,
                                        truncation=False,
                                        return_tensors="pt")

            length_of_input_encoding = len(test_input_encoding['input_ids'][0])


            if length_of_input_encoding > self.max_len_input:
              self.skippedcount = self.skippedcount + 1
              continue

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input, pad_to_max_length=True, return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output, pad_to_max_length=True,return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)


In [ ]:
train_dataset = QuestionGenerationDataset(t5_tokenizer,train_file_path)
validation_dataset = QuestionGenerationDataset(t5_tokenizer,validation_file_path)
train_sample = train_dataset[51]
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print (decoded_train_input)
print (decoded_train_output)

In [ ]:

class T5FineTuner(pl.LightningModule):
    def __init__(self,hparams, t5model, t5tokenizer):
        super(T5FineTuner, self).__init__()
        self.hparams = hparams
        self.model = t5model
        self.tokenizer = t5tokenizer


    def forward( self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )

         return outputs


    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.hparams.batch_size,num_workers=4)

    def val_dataloader(self):
        return DataLoader(validation_dataset, batch_size=self.hparams.batch_size,num_workers=4)



    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer


In [ ]:
args_dict = dict(
    batch_size=4,
)

args = argparse.Namespace(**args_dict)


model = T5FineTuner(args,t5_model,t5_tokenizer)

trainer = pl.Trainer(max_epochs = 1, gpus=1,progress_bar_refresh_rate=30)

trainer.fit(model)

print ("Saving model")
save_path_model = '/content/gdrive/My Drive/Colab Notebooks/t5/model/'
save_path_tokenizer = '/content/gdrive/My Drive/Colab Notebooks/t5/tokenizer/'
model.model.save_pretrained(save_path_model)
t5_tokenizer.save_pretrained(save_path_tokenizer)

In [ ]:
from transformers import T5ForConditionalGeneration,T5Tokenizer
trained_model_path = '/content/gdrive/My Drive/Colab Notebooks/t5/model/'
trained_tokenizer = '/content/gdrive/My Drive/Colab Notebooks/t5/tokenizer/'
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)
context = df2['sentences_new']
text = "context: "+context


In [ ]:
input_ids =[]
attention_mask = []
for x in (text):
  encoding = tokenizer.encode_plus(x,max_length =512, padding=True, return_tensors="pt")
  print (encoding.keys())
  input_ids.append(encoding["input_ids"].to(device))
  attention_mask.append(encoding["attention_mask"].to(device))
  input_ids, attention_mask = encoding["input_ids"].to(device),encoding["attention_mask"].to(device)
data = dict(zip(input_ids, attention_mask))


model.eval()
cqs = []

for x in (text):
  encoding = tokenizer.encode_plus(x,max_length =512, padding=True, return_tensors="pt")
  print (encoding.keys())
  input_ids, attention_mask = encoding["input_ids"].to(device),encoding["attention_mask"].to(device)
  beam_outputs = model.generate(
      input_ids=input_ids,attention_mask=attention_mask,
      max_length=72,
      early_stopping=True,
      num_beams=5,
      num_return_sequences=3
  )

  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      cqs.append(sent)





In [ ]:
questions =[]
with open("CQs.txt", "w") as file:
    file.write(str(cqs))
with open("./CQs.txt","r",encoding="utf-8") as f:
  for line in f.readlines():
    questions.append(line)
new_questions = []
for i in (questions):
  x = str(i).split(":")
  x = str(i).replace("question", '')
  new_questions.append(x)
new_questions2 = str(new_questions).split(",")
dcqs =pd.DataFrame(new_questions2)
dcqs.columns = ['CQs']
dcqs['CQs'] = dcqs['CQs'].map(lambda x: x.replace(":", '').replace("'", '').replace('[', '').replace(']', '').replace('"', ''))
dcqs.head()


new_questions = []
for i in (questions):
  x = str(i).split(":")
  x = str(i).replace("question", '')
  new_questions.append(x)
new_questions2 = str(new_questions).split(",")
dcqs =pd.DataFrame(new_questions2)
dcqs.columns = ['CQs']
dcqs['CQs'] = dcqs['CQs'].map(lambda x: x.replace(":", '').replace("'", '').replace('[', '').replace(']', '').replace('"', ''))
dcqs.head()
#new_questions

for i in range(0, len(dcqs)):
    dcqs.at[i, 'newCQs'] =re.sub(r"\s\([A-Z][a-z]+,\s[A-Z][a-z]?\.[^\)]*,\s\d{4}\)","",str(dcqs.loc[i,'CQs']))
with open("output.txt", "w") as f:
    f.write("\n".join(dcqs["newCQs"]))

In [ ]:
#write dataframe to CSV
#RUN CANDIDATE PATTERN CODE
#IMPORT CLaRO DATA
# ITERATE TO SEE HOW MANY PATTERS ARE PRESENT IN CLaRO
# RUN STATS DISTRIBUTION ON THE THE TYPE OF QUESTIONS GENERATED

In [ ]:
#Code modiefied slightly, originaly by Linguistics pattern extraction team
# #CREATE  PATTERNS FROM CQs
def mark_chunk(cq, spans, chunktype, offset, counter):
    for (start, end) in spans:  # for each span of EC/PC candidate
        cq = cq[:start - offset] + chunktype + str(counter) +\
            cq[end - offset:]  # substitute that candidate with EC/PC marker
        offset += (end - start) - len(chunktype) - 1  # check by how much the total length of CQ changed
    return cq, offset


def extract_EC_chunks(cq):
    """
        Find EC chunks and replace their occurences with EC tags
    """

    def _get_EC_span_reject_wh_starters(chunk):
        """
            By default, SpaCy treats question words (wh- pronouns starting questions: where, what,...)
            as nouns, so often if questions starts with wh- pronoun + noun (like: What software is the best?)
            the whole "what software" is interpreted as EC chunk - this function tries to fix that issue by
            omitting wh- word if EC candidate consists of multiple tokens, and first token in question is wh- word.
            The same issue occurs for "How" starter.
            Moreover, chunks extracted with SpaCy enclose words like 'any', 'some' which are important for us, so
            they shouldn't be substituted with 'EC' marker. Thus we remove such words if they prepend the EC.
            The result is returned as the span - the position of beginning of the fixed EC and position of end.
        """
        if (len(chunk) > 1 and
                (chunk[0].text.lower().startswith("wh")) or
                (chunk[0].text.lower() == 'how')):
            chunk = chunk[1:]

        if chunk[0].text.lower() in ['any', 'some', 'many', 'well', 'its']:
            chunk = chunk[1:]

        return (chunk.start_char, chunk.end_char)

    doc = nlp(cq)

    #  thins classified as ECs which shouldn't be interpreted that way
    rejecting_ec = ["what", "which", "when", "where", "who", "type", "types",
                    "kinds", "kind", "category", "categories", "difference",
                    "differences", "extent", "i", "we", "respect", "there",
                    "not", "the main types", "the possible types", "the types",
                    "the difference", "the differences", "the main categories"]

    counter = 1  # counter indicating current chunk id (EC1, EC2, ...)
    offset = 0   # if we replace for example "Weka" with 'EC1' then the new CQ will be shorter by one char the offset remembers by how much we have shortened the current CQ with EC markers, so the new substitutions can be applied in correct places.


    # we decided to treat qualities defined as adjectives in: How + Quality(adjective) + Verb as EC
    if (doc[0].text.lower() == 'how' and
            doc[1].pos_ == 'ADJ' and
            doc[2].pos_ == 'VERB'):

        start = doc[1].idx  # mark where quality starts
        end = start + len(doc[1])  # mark where quality ends

        cq, offset = mark_chunk(cq, [(start, end)], "EC", offset, counter)  # substitute quality with EC identifier
        counter += 1  # the next EC chunk should have a new, bigger identifier, for example EC2

    for chunk in doc.noun_chunks:  # for each EC chunk candidate detected
        (start, end) = _get_EC_span_reject_wh_starters(chunk)  # check where chunk begins and ends

        ec = cq[start - offset:end - offset]  # extract text of potential EC, apply offsets

        if ec.lower() in rejecting_ec:  # if it should be rejected - do nothing
            continue

        if "the thing" in ec and end - start > len("the thing"):
            cq = cq[:start - offset] + "EC" + str(counter) +\
                " EC" + str(counter + 1) + cq[end - offset:]
            counter += 2
            offset += (end - start) - 7
        else:
            cq, offset = mark_chunk(cq, [(start, end)], "EC", offset, counter)
            counter += 1

    if (doc[-2].pos_ == 'VERB' and doc[-3].text in ['are', 'is', 'were', 'was'] and doc[-1].text == '?') or (doc[-2].pos_ in ['ADJ', 'ADV'] and doc[-1].text == '?'):
        # if CQ ends with are/is/were/was + VERB + ? or the last token is ADJective or ADVerb, treat the
        # verb / adverb / adjective as EC
        # Which animals are endangered -> endangered is EC
        # Which animals are quick -> quick is EC
        if doc[-2].text.lower() not in rejecting_ec:
            start = doc[-2].idx
            end = start + len(doc[-2])

            cq, offset = mark_chunk(
                cq, [(start, end)], "EC", offset, counter)
            counter += 1

    return cq


def get_PCs_as_spans(cq):
    def _is_auxilary(token, chunk_token_ids):
        """
            Check if given token is an auxilary verb of detected PC.
            The auxilary verb can be in a different place than the main part
            of the PC, so pos-tag-sequence based rules don't work here.
            For example in "What system does Weka require?" - the main part
            of PC is the word 'required'. The auxilary verb 'does' is separeted
            from the main part by 'Weka' noun. Thus dependency tree is used
            to identify auxilaries.
        """
        if (token.head.i in chunk_token_ids and  # if dep-tree current token's parent (head) is somewhere inside the main part of PC represented as chunk_token_ids (sequence of numeric token identifiers)
                token.dep_ == 'aux' and  # if the dep-tree label on the edge between some word from main part of PC and current token is AUX (auxilary)
                token.i not in chunk_token_ids):  # if token is outside of detected main part of PC
            return True  # yep, it's auxilary
        else:
            return False

    def _get_span(group, doc):
        id_tags = group.split(",")
        ids = [int(id_tag.split("::")[0]) for id_tag in id_tags]
        aux = None
        for token in doc:
            if _is_auxilary(token, ids):
                aux = token

        return (doc[ids[0]].idx, doc[ids[-1]].idx + len(doc[ids[-1]]),
                aux)

    def _reject_subspans(spans):
        """
            Given list of (chunk begin index, chunk end index) spans,
            return only those spans that aren't subspans of any other span.
            For instance form list [(1,10), (2,5)], the second span
            will be rejected because it is a subspan of the first one.
        """
        filtered = []
        for i, span in enumerate(spans):
            subspan = False
            for j, other in enumerate(spans):
                if i == j:
                    continue

                if span[0] >= other[0] and span[1] <= other[1]:
                    subspan = True
                    break
            if subspan is False:
                filtered.append(span)
        return filtered

    doc = nlp(cq)

    """
        Transform CQ into a form of POS-tags with token sequence identifier.
        Each token is described with "{ID}::{POS_TAG}".
        Tokens are separated with ","
        Having that form, we can extract longest sequences of expected pos-tags
        using regexes. The extracted parts can be explored to collect identifiers
        of tokens, so we know where they are located in text.
        Ex: "Kate owns a cat" should be translated into: "1::NOUN,2::VERB,3::DET,4::NOUN"
    """
    pos_text = ",".join(
        ["{id}::{pos}".format(id=id, pos=t.pos_) for id, t in enumerate(doc)])

    regexes = [   # rules describing PCs
        r"([0-9]+::(PART|VERB),?)*([0-9]+::VERB)",
        r"([0-9]+::(PART|VERB),?)+([0-9]+::AD(J|V),)+([0-9]+::ADP)",
        r"([0-9]+::(PART|VERB),?)+([0-9]+::ADP)",
    ]

    spans = []  # list of beginnings and endings of each chunk
    for regex in regexes:  # try to extract chunks with regexes
        for m in re.finditer(regex, pos_text):
            spans.append(_get_span(m.group(), doc))  # get chunk begin and end if matched
    spans = _reject_subspans(spans)  # reject subspans
    return spans


def extract_PC_chunks(cq):
    rejecting_pc = ['is', '痴', 'are', 'was', 'do', 'does', 'did', 'were',
                    'have', 'had', 'can', 'could', 'categorise', 'regarding',
                    'is of', 'are of', 'are in', 'given', 'is there']

    offset = 0
    counter = 1

    for begin, end, aux in get_PCs_as_spans(cq):
        if cq[begin - offset:end - offset].lower() in rejecting_pc:
            continue

        spans = [(begin, end)]

        if aux:
            spans.insert(0, (aux.idx, aux.idx + len(aux)))

        cq, offset = mark_chunk(cq, spans, "PC", offset, counter)
        counter += 1

    return cq


# def mark_placeholders(cq):
#     """
#         In our paper we stated that if a CQ is dematerialized (= it contains
#         placeholders represented with [] that can be filled in various ways),
#         the whole dematerialized
#         placeholder should be treated as an EC. Thus we transform the whole
#         placholder (everything in []) into the string 'the thing' (*).
#         (*) One can imagine a nicer way of solving it, but this approach works
#     """
#     return re.sub(r'\[.*?\]', 'the thing', cq)


# def clean_cq(cq):
#     normalizers = {
#         "[-\"\'脳": "",  # remove all dashes and quotations
#         "\(.*?\)": "",   # remove all ( )
#         "[ \t]+": " ",   # remove all multiplied whitespaces
#         " \?": "?"       # remove whitespaces before ?
#     }

#     for regex in normalizers:
#         cq = re.sub(regex, normalizers[regex], cq)
#     return cq




# def generate(cq):
#   patterns = set()
#   pattern_candidates = {}
#   T5Cqs_patterns = []
#   with open("./output.txt", 'r') as f:
#     for line in f:
#       line = line.split("\t")[0].strip()
#       cq = clean_cq(line.strip())
#       #pattern = mark_placeholders(cq)
#       pattern = extract_EC_chunks(cq)
#       pattern = extract_PC_chunks(cq)

#       patterns.add(pattern)
#       pattern_candidates.update(pattern:cq)

#   for elem in patterns:
#     #print(elem)
#     T5Cqs_patterns.append(elem)
#   with open("CQs_patterns.txt", "w") as file:
#     file.write(str(T5Cqs_patterns))
#     createdPatternsDF = pd.DataFrame([pattern_candidates], columns=['patterns', 'questions'])
#   return createdPatternsDF, T5Cqs_patterns




In [ ]:
genQuestion=[] 
splitQuestions =[]
patterns = []
pattern_candidates = {}
T5Cqs_patterns = []
path_to_output = "./output.txt"
with open(path_to_output, 'r') as f:
  for line in f.readlines():
    line= line.replace('?', '? ')
    line2 = line.split("?")
    line3= [p.join(' ?') for p in line2]#line2.re.replace(',', '?,')
    line3 = [p.lstrip() for p in line3]
    genQuestion.append(line3)

In [ ]:
genQuestionDF= pd.DataFrame(genQuestion).T
genQuestionDF.columns=['questions']

allList=[]
pat_candidates={}
for sublist in  genQuestion: 
  final = []
  for i in sublist:
    if(i==''):
      continue
    for item in i.split(','):
      pat = extract_EC_chunks(item)
      #print(pat)
     # break
      pat = extract_PC_chunks(pat)
      final.append(pat)
      #pat_candidates[item]=[]
      pat_candidates[item]=(pat)
      
  allList.append(final)

In [ ]:
abCQs= pd.DataFrame(pat_candidates.items(), columns=['questions', 'patterns'])
abCQs = abCQs.iloc[:-1]


In [ ]:
#import templates as pandas df and also put patterns in another dataframe.
#next compare to see those in template and place in a new column
def compare_patterns(abCQs):
 
  templates= pd.read_csv("./CLaROv2.csv", sep = ",")
  templates.columns = 'ID', 'question_patterns'
  check = pd.merge(df1, templates, on=['question_patterns'], how='left', indicator='Exist')
  check_both = check[check['Exist'] == 'both']
  check_cqPatterns_only = check[check['Exist'] == 'left_only']
  check_templates_only = check[check['Exist'] == 'right_only']
  distinct_templates = check_both.drop_duplicates(subset='patterns', keep="first")
  return check_both, check_cqPatterns_only, check_templates_only, distinct_templates
  


In [ ]:
# model.eval()
# for i, j in (data):
#   beam_outputs = model.generate(
#       input_ids= data[i], attention_mask= data[j],
#       max_length=72,
#       early_stopping=True,
#       no_repeat_ngram_size=2,
#       num_beams=5,
#       num_return_sequences=3
#   )

#   cqs = []
#   #for i, j in (input_ids, attention_mask):
#   for beam_output in beam_outputs:
#     sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
#     cqs.append(sent)
# questions =[]
# with open("CQs.txt", "w") as file:
#     file.write(str(cqs))
# with open("./CQs.txt","r",encoding="utf-8") as f:
#   for line in f.readlines():
#    #full_text = f.read()
#    #l = extract_sentences(full_text)
#     #for l in re.split(r"('\.')", full_text):
#    questions.append(line)
# df =pd.DataFrame(questions).T
# df.head()